In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import os
import sqlite3

# Preprocessing
#from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler

# Model
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
import optuna
import shap

from tempfile import mkdtemp
# Create a temp folder
cachedir = mkdtemp()

In [2]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data')

# 1. Import processed data

In [3]:
df_data = pd.read_csv(os.path.join(DATA_PATH,'processed_entsoe_data.csv'))

## Missing values

In [4]:
from scripts.preprocess import missing_data

df_preproc = missing_data(df_data)


Missing Data Removal Summary:
--------------------------------------------------
Initial number of rows: 86752

Step 1: Generation Data
-------------------------
DE: 2,176 rows removed
BE: 40 rows removed
CH: 40 rows removed
IT: 64 rows removed
ES: 63 rows removed
FR: 61 rows removed
Rows after generation cleaning: 84,509
Total rows removed: 2,243

Step 2: Price Data
-------------------------
Rows removed: 72
Rows remaining: 84,437

Step 3: Load Data
-------------------------
Rows removed: 81
Final number of rows: 84,356

Total rows removed: 2,396
Percentage of data retained: 97.2%


## Feature engineering

In [5]:
from scripts.preprocess import feature_engineering

df_preproc = feature_engineering(df_preproc)

Adding cyclical time features...
Converting datetime to index...
Creating cyclical features...
Feature engineering completed!


## Merge with fuel prices

In [6]:
from scripts.preprocess import merge_fuel_prices

df_preproc = merge_fuel_prices(df_preproc)


Processing fuel and carbon prices...
Loading EUA carbon prices...
Loading TTF gas prices...
Loading ARA coal prices...
Loading USD/EUR exchange rates...
Merging price datasets...
Converting coal prices to EUR...
Processing dates for monthly merging...
Saving processed fuel prices to: /home/ronan/code/Power_Price_Forecasting/data/fuel_prices.csv
Merging with main dataset...
Fuel price merging completed!


## Split train test

### For now we do not care about the chronology

In [10]:
df_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84356 entries, 0 to 84355
Data columns (total 95 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   BE_Biomass                           84356 non-null  float64
 1   DE_Biomass                           84356 non-null  float64
 2   ES_Biomass                           84356 non-null  float64
 3   FR_Biomass                           84347 non-null  float64
 4   IT_Biomass                           84331 non-null  float64
 5   BE_Fossil Brown coal/Lignite         12 non-null     float64
 6   DE_Fossil Brown coal/Lignite         84356 non-null  float64
 7   ES_Fossil Brown coal/Lignite         84356 non-null  float64
 8   BE_Fossil Gas                        84356 non-null  float64
 9   CH_Fossil Gas                        6121 non-null   float64
 10  DE_Fossil Gas                        84356 non-null  float64
 11  ES_Fossil Gas               

In [13]:
X = df_preproc.drop(columns=['FR_price'])
y = df_preproc['FR_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [14]:
pipe = Pipeline([
    ('imputer_zero', SimpleImputer(fill_value=0)),
    ('robustscaler', RobustScaler())]
    ).set_output(transform='pandas')

excluded = ['Day_sin', 'Day_cos', 'Year_sin', 'Year_cos']
selected_columns = [col for col in X_train.columns if col not in excluded]

preproc_pipeline = ColumnTransformer([('base_pipeline', pipe, selected_columns)], remainder='passthrough').set_output(transform='pandas')

preproc_pipeline

,transformers,"[('base_pipeline', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,0


In [15]:
X_train_preproc = preproc_pipeline.fit_transform(X_train)

X_train_preproc

,base_pipeline__BE_Biomass,base_pipeline__DE_Biomass,base_pipeline__ES_Biomass,base_pipeline__FR_Biomass,base_pipeline__IT_Biomass,base_pipeline__BE_Fossil Brown coal/Lignite,base_pipeline__DE_Fossil Brown coal/Lignite,base_pipeline__ES_Fossil Brown coal/Lignite,base_pipeline__BE_Fossil Gas,base_pipeline__CH_Fossil Gas,...,base_pipeline__DE_price,base_pipeline__ES_price,base_pipeline__IT_price,base_pipeline__EUA_EUR,base_pipeline__TTF_EUR,base_pipeline__ARA_EUR,remainder__Day_sin,remainder__Day_cos,remainder__Year_sin,remainder__Year_cos
63538,0.034884,-1.056325,0.785124,-0.322581,0.858696,0.0,-0.134665,0.000000,-0.576358,0.0,...,2.959048,2.193541,5.827329,0.887476,9.449813,5.225258,0.707107,7.071068e-01,-0.578976,-0.815345
51131,-0.067442,-0.443675,0.553719,0.435484,0.880435,0.0,0.140200,0.000000,0.442173,0.0,...,0.006293,-0.228010,-0.125188,0.276256,-0.054683,-0.222818,0.258819,9.659258e-01,0.864304,0.502970
12628,0.281395,0.415974,-0.347107,-0.983871,-0.804348,0.0,-0.026584,0.000000,-0.355272,0.0,...,-0.156786,-0.159964,-0.385363,-0.344749,-0.253410,-0.407139,0.965926,2.588190e-01,0.197689,-0.980265
28880,0.006977,0.470914,-0.132231,-0.241935,-0.184783,0.0,0.281229,0.000000,-0.267093,0.0,...,-0.055545,0.099727,-0.058260,-0.174494,0.100460,0.163970,-0.965926,2.588190e-01,0.815869,-0.578236
59958,0.095349,0.518006,1.090909,2.129032,1.119565,0.0,-0.029112,0.000000,0.348243,0.0,...,8.847410,11.298343,7.141770,0.829583,4.545067,3.147713,-0.965926,2.588190e-01,0.921884,0.387467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,0.327907,0.207295,-0.785124,0.661290,-0.510870,0.0,0.358977,2.717172,0.313738,0.0,...,0.018333,0.029226,0.062330,-0.335616,-0.233404,-0.051708,1.000000,-8.428608e-13,0.201075,-0.979576
45891,0.174419,-0.497230,0.024793,0.225806,0.326087,0.0,-0.508983,0.000000,1.170607,0.0,...,-0.243068,-0.248536,-0.277736,0.010926,-0.612744,-0.536085,-0.258819,-9.659258e-01,-0.366218,-0.930529
42613,0.174419,0.413666,0.198347,-0.274194,0.282609,0.0,-1.027205,0.000000,-0.067093,0.0,...,-0.330445,-0.508450,-0.431489,-0.129322,-0.574926,-0.490860,-0.500000,8.660254e-01,0.912356,0.409398
43567,0.118605,0.056325,-0.471074,-0.080645,0.532609,0.0,-1.086955,0.000000,-0.432588,0.0,...,-0.249453,-0.905349,-0.530072,-0.098500,-0.604225,-0.642333,-0.866025,-5.000000e-01,0.965870,-0.259028


# 2. Forecast with XGBoost

## First model

In [20]:
xgb_reg = XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.1)

xgb_pipe = make_pipeline(preproc_pipeline, xgb_reg, memory=cachedir)

# Cross-validate using the MSE scorer
xgb_score = cross_val_score(
    xgb_pipe, 
    X_train, 
    y_train, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1,
    verbose=1).mean()

np.sqrt(-xgb_score)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_res

np.float64(13.581972759230371)

## Hyperparameters tuning

In [21]:
xgb_pipe.get_params()

{'memory': '/tmp/tmpdncbub_z',
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('base_pipeline',
                                    Pipeline(steps=[('imputer_zero',
                                                     SimpleImputer(fill_value=0)),
                                                    ('robustscaler',
                                                     RobustScaler())]),
                                    ['BE_Biomass', 'DE_Biomass', 'ES_Biomass',
                                     'FR_Biomass', 'IT_Biomass',
                                     'BE_Fossil Brown coal/Lignite',
                                     'DE_Fossil Brown coal/Lignite',
                                     'ES_Fossil Brown coal/Lignite',
                                     'BE_Fossil Gas', 'CH_Fossil Gas'...
                                     'ES_Fossil Hard coal', 'FR_Fossil Hard coal',
                                     

In [22]:
def objective(trial, X, y, pipeline):
    # Define the hyperparameter search space
    params = {
        'xgbregressor__max_depth': trial.suggest_int('xgbregressor__max_depth', 3, 10),
        'xgbregressor__learning_rate': trial.suggest_float('xgbregressor__learning_rate', 0.01, 0.3, log=True),
        'xgbregressor__n_estimators': trial.suggest_int('xgbregressor__n_estimators', 100, 1000),
        'xgbregressor__min_child_weight': trial.suggest_int('xgbregressor__min_child_weight', 1, 7),
        'xgbregressor__subsample': trial.suggest_float('xgbregressor__subsample', 0.6, 1.0),
        'xgbregressor__colsample_bytree': trial.suggest_float('xgbregressor__colsample_bytree', 0.6, 1.0),
        'xgbregressor__gamma': trial.suggest_float('xgbregressor__gamma', 0, 5),
        'xgbregressor__reg_alpha': trial.suggest_float('xgbregressor__reg_alpha', 0, 5),
        'xgbregressor__reg_lambda': trial.suggest_float('xgbregressor__reg_lambda', 0, 5)
    }
    
    # Set the parameters in the pipeline
    pipeline.set_params(**params)
    print(pipeline)
    
    # Perform cross-validation
    cv_scores = cross_val_score(
        pipeline,
        X,
        y,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    return np.mean(cv_scores)

In [24]:
study = optuna.create_study(direction='maximize',study_name='xgb_optim', sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(lambda trial: objective(trial, X_train, y_train, xgb_pipe), n_trials=50)
best_params = study.best_params

[I 2025-07-29 20:32:30,916] A new study created in memory with name: xgb_optim


Pipeline(memory='/tmp/tmpdncbub_z',
         steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('base_pipeline',
                                                  Pipeline(steps=[('imputer_zero',
                                                                   SimpleImputer(fill_value=0)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['BE_Biomass', 'DE_Biomass',
                                                   'ES_Biomass', 'FR_Biomass',
                                                   'IT_Biomass',
                                                   'BE_Fossil Brown '
                                                   'coal/Lignite',
                                                   'DE_Fossil Brown '
                    

/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from usi

Pipeline(memory='/tmp/tmpdncbub_z',
         steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('base_pipeline',
                                                  Pipeline(steps=[('imputer_zero',
                                                                   SimpleImputer(fill_value=0)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['BE_Biomass', 'DE_Biomass',
                                                   'ES_Biomass', 'FR_Biomass',
                                                   'IT_Biomass',
                                                   'BE_Fossil Brown '
                                                   'coal/Lignite',
                                                   'DE_Fossil Brown '
                    

/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from usi

...

[I 2025-07-29 20:47:05,346] Trial 47 finished with value: -179.97291326474243 and parameters: {'xgbregressor__max_depth': 4, 'xgbregressor__learning_rate': 0.08780688458132625, 'xgbregressor__n_estimators': 541, 'xgbregressor__min_child_weight': 7, 'xgbregressor__subsample': 0.626121682860712, 'xgbregressor__colsample_bytree': 0.9132937753255252, 'xgbregressor__gamma': 1.4419924866574696, 'xgbregressor__reg_alpha': 1.20709310038287, 'xgbregressor__reg_lambda': 3.31252285766338}. Best is trial 4 with value: -155.40686631763012.


Pipeline(memory='/tmp/tmpdncbub_z',
         steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('base_pipeline',
                                                  Pipeline(steps=[('imputer_zero',
                                                                   SimpleImputer(fill_value=0)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['BE_Biomass', 'DE_Biomass',
                                                   'ES_Biomass', 'FR_Biomass',
                                                   'IT_Biomass',
                                                   'BE_Fossil Brown '
                                                   'coal/Lignite',
                                                   'DE_Fossil Brown '
                    

[I 2025-07-29 20:47:13,580] Trial 48 finished with value: -172.73299595837938 and parameters: {'xgbregressor__max_depth': 4, 'xgbregressor__learning_rate': 0.0962841177215152, 'xgbregressor__n_estimators': 566, 'xgbregressor__min_child_weight': 3, 'xgbregressor__subsample': 0.8218751234645676, 'xgbregressor__colsample_bytree': 0.7148206079678519, 'xgbregressor__gamma': 3.5328735313648947, 'xgbregressor__reg_alpha': 2.0742843466678202, 'xgbregressor__reg_lambda': 1.8027278024294613}. Best is trial 4 with value: -155.40686631763012.


Pipeline(memory='/tmp/tmpdncbub_z',
         steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('base_pipeline',
                                                  Pipeline(steps=[('imputer_zero',
                                                                   SimpleImputer(fill_value=0)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['BE_Biomass', 'DE_Biomass',
                                                   'ES_Biomass', 'FR_Biomass',
                                                   'IT_Biomass',
                                                   'BE_Fossil Brown '
                                                   'coal/Lignite',
                                                   'DE_Fossil Brown '
                    

[I 2025-07-29 20:47:23,952] Trial 49 finished with value: -175.45553991248198 and parameters: {'xgbregressor__max_depth': 9, 'xgbregressor__learning_rate': 0.23242791456070774, 'xgbregressor__n_estimators': 141, 'xgbregressor__min_child_weight': 2, 'xgbregressor__subsample': 0.7394077477970253, 'xgbregressor__colsample_bytree': 0.9259865917480989, 'xgbregressor__gamma': 4.927457138216488, 'xgbregressor__reg_alpha': 4.844858523351759, 'xgbregressor__reg_lambda': 4.5247417277496345}. Best is trial 4 with value: -155.40686631763012.


In [30]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [31]:
optuna.visualization.plot_param_importances(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

## Cross-val check score

In [34]:
xgb_pipe.set_params(**study.best_params)

# Cross-validate using the MSE scorer
xgb_score = cross_val_score(
    xgb_pipe, 
    X_train, 
    y_train, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1,
    verbose=1).mean()
np.sqrt(-xgb_score)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ronan/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_res

np.float64(12.466229033578282)